In [2]:
!pip install hydra-core

     |████████████████████████████████| 133kB 12.9MB/s 
     |████████████████████████████████| 112kB 14.4MB/s 
     |████████████████████████████████| 645kB 19.0MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-cp37-none-any.whl size=141231 sha256=3cf2bfddcd59e6b907fdc8744b19b6214d2965b42a3025cb4c9eca37ff6d8ddc
  Stored in directory: /root/.cache/pip/wheels/e3/e2/fa/b78480b448b8579ddf393bebd3f47ee23aa84c89b6a78285c8
Successfully built antlr4-python3-runtime
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [1]:
import torch

In [3]:
bart = torch.hub.load('pytorch/fairseq', 'bart.base')
bart.eval()

Downloading: "https://github.com/pytorch/fairseq/archive/master.zip" to /root/.cache/torch/hub/master.zip


running build_ext
cythoning fairseq/data/data_utils_fast.pyx to fairseq/data/data_utils_fast.cpp


/usr/local/lib/python3.7/dist-packages/torch/utils/cpp_extension.py:369: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))


cythoning fairseq/data/token_block_utils_fast.pyx to fairseq/data/token_block_utils_fast.cpp
building 'fairseq.libbleu' extension
creating build
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/fairseq
creating build/temp.linux-x86_64-3.7/fairseq/clib
creating build/temp.linux-x86_64-3.7/fairseq/clib/libbleu
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-a56wZI/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-a56wZI/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.7m -c fairseq/clib/libbleu/libbleu.cpp -o build/temp.linux-x86_64-3.7/fairseq/clib/libbleu/libbleu.o -std=c++11 -O3 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE="_gcc" -DPYBIND11_STDLIB="_libstdcpp" -DPYBIND11_BUILD_ABI="_cxxabi1011" -DTORCH_EXTENSION_

100%|██████████| 422876945/422876945 [00:51<00:00, 8232763.45B/s] 
1042301B [00:00, 1227161.50B/s]
456318B [00:00, 660514.46B/s]


BARTHubInterface(
  (models): ModuleList(
    (0): BARTModel(
      (encoder): TransformerEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(51201, 768, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(1026, 768, padding_idx=1)
        (layernorm_embedding): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0): TransformerEncoderLayer(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout_module): FairseqDropout()
       

In [21]:
x = bart.fill_mask(["Gina's friend Tami had a folder that Gina wanted. And since <mask> So she decided she would try to find the folder on her own."], topk=3, beam=10, match_source_len=False)

In [35]:
x[0]

[("Gina's friend Tami had a folder that Gina wanted. And since Gina didn't have it, she couldn't find it. So she decided she would try to find the folder on her own.",
  tensor(-0.2027)),
 ("Gina's friend Tami had a folder that Gina wanted. And since Gina didn't have a folder, she couldn't find it. So she decided she would try to find the folder on her own.",
  tensor(-0.2095)),
 ("Gina's friend Tami had a folder that Gina wanted. And since Gina didn't have the folder, she couldn't find it. So she decided she would try to find the folder on her own.",
  tensor(-0.2116))]

In [39]:
x[0][1][0]

"Gina's friend Tami had a folder that Gina wanted. And since Gina didn't have a folder, she couldn't find it. So she decided she would try to find the folder on her own."

In [43]:
val_lines_list = []
val_file = open('val.source')
val_lines = val_file.readlines()
for line in val_lines:
  val_lines_list.append(line)

In [45]:
val_lines_list2 = []
for line in val_lines_list:
  insert_str = "And since <mask>."
  line_blocks = line.split("#")
  if (len(line_blocks) == 2): # if there is only one #
    new_str = line_blocks[0] + insert_str + line_blocks[1]
    val_lines_list2.append(new_str)

In [47]:
len(val_lines_list2)

14313

In [48]:
print(56/14313)

0.003912527073290016


In [52]:
target_lines = []
counter = 0
limit = 1000
for masked_line in val_lines_list2:
  if (counter < limit):
    x = bart.fill_mask([masked_line], topk=3, beam=10, match_source_len=False)
    target_lines.append(x[0][0][0])
    target_lines.append(x[0][1][0])
    target_lines.append(x[0][2][0])
    counter += 1
    print(str(counter) + " / " + str(limit))
  else:
    break

1 / 1000
2 / 1000
3 / 1000
4 / 1000
5 / 1000
6 / 1000
7 / 1000
8 / 1000
9 / 1000
10 / 1000
11 / 1000
12 / 1000
13 / 1000
14 / 1000
15 / 1000
16 / 1000
17 / 1000
18 / 1000
19 / 1000
20 / 1000
21 / 1000
22 / 1000
23 / 1000
24 / 1000
25 / 1000
26 / 1000
27 / 1000
28 / 1000
29 / 1000
30 / 1000
31 / 1000
32 / 1000
33 / 1000
34 / 1000
35 / 1000
36 / 1000
37 / 1000
38 / 1000
39 / 1000
40 / 1000
41 / 1000
42 / 1000
43 / 1000
44 / 1000
45 / 1000
46 / 1000
47 / 1000
48 / 1000
49 / 1000
50 / 1000
51 / 1000
52 / 1000
53 / 1000
54 / 1000
55 / 1000
56 / 1000
57 / 1000
58 / 1000
59 / 1000
60 / 1000
61 / 1000
62 / 1000
63 / 1000
64 / 1000
65 / 1000
66 / 1000
67 / 1000
68 / 1000
69 / 1000
70 / 1000
71 / 1000
72 / 1000
73 / 1000
74 / 1000
75 / 1000
76 / 1000
77 / 1000
78 / 1000
79 / 1000
80 / 1000
81 / 1000
82 / 1000
83 / 1000
84 / 1000
85 / 1000
86 / 1000
87 / 1000
88 / 1000
89 / 1000
90 / 1000
91 / 1000
92 / 1000
93 / 1000
94 / 1000
95 / 1000
96 / 1000
97 / 1000
98 / 1000
99 / 1000
100 / 1000
101 / 10

In [56]:
output_file = open("enthymeme.hypo", "w")
output_file.writelines(target_lines)